In [1]:
import zipfile
import requests
import io

### Данные ###

In [2]:
url = "https://www.dropbox.com/s/k23enjvr3fb40o5/corpora.zip?dl=1"
    
r = requests.get(url, stream=True)

with zipfile.ZipFile(io.BytesIO(r.content)) as zip_ref:
    zip_ref.extractall("data/.")

### Задание 1 ###
Реализуйте базовый частотный метод по Шерлоку Холмсу:

- подсчитайте частоты букв по корпусам (пунктуацию и капитализацию можно просто опустить, а вот пробелы лучше оставить);
- возьмите какие-нибудь тестовые тексты (нужно взять по меньшей мере 2-3 предложения, иначе вряд ли сработает), зашифруйте их посредством случайной перестановки символов;
- расшифруйте их таким частотным методом.

In [11]:
with open('data/WarAndPeaceEng.txt', 'r') as iofile:
    text = iofile.readlines()
    
TEXT_START = 806
TEXT_END = 64649
text = text[TEXT_START : TEXT_END]

In [12]:
text = ''.join(str(x).lower() for x in text)
out = [token for token in text]
tokens = sorted(set(out))
print(len(tokens))
print(tokens)

57
['\n', ' ', '!', '"', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\xa0', '¤', '©', 'г', 'є']


In [13]:
bad_chars = ['!', '"', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '=', '?', '\xa0', '¤', '©', 'г', 'є']
text = ''.join(i for i in text if not i in bad_chars)
text = text.replace("\n", " ")
out = [token for token in text]
tokens = sorted(set(out))
print(len(tokens))
print(tokens)

27
[' ', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [14]:
frequencies_dict = {}
for token in text:
    frequencies_dict[token] = frequencies_dict.get(token, 0) + 1

print(frequencies_dict)

{'w': 58929, 'e': 313022, 'l': 96034, ' ': 575536, 'p': 44715, 'r': 146888, 'i': 172639, 'n': 183127, 'c': 60657, 's': 162126, 'o': 191486, 'g': 50909, 'a': 204415, 'd': 117752, 'u': 64919, 'j': 2485, 't': 224507, 'f': 54507, 'm': 61283, 'y': 45934, 'h': 166519, 'b': 34369, 'v': 26788, 'k': 20281, 'x': 4032, 'z': 2386, 'q': 2319}


In [15]:
from collections import Counter

frequencies_dict_counter = Counter(frequencies_dict)
print(frequencies_dict_counter.most_common())
frequencies_dict_normalized = [(item[0], item[1]/len(text)) for item in frequencies_dict_counter.most_common()]
print(frequencies_dict_normalized)

[(' ', 575536), ('e', 313022), ('t', 224507), ('a', 204415), ('o', 191486), ('n', 183127), ('i', 172639), ('h', 166519), ('s', 162126), ('r', 146888), ('d', 117752), ('l', 96034), ('u', 64919), ('m', 61283), ('c', 60657), ('w', 58929), ('f', 54507), ('g', 50909), ('y', 45934), ('p', 44715), ('b', 34369), ('v', 26788), ('k', 20281), ('x', 4032), ('j', 2485), ('z', 2386), ('q', 2319)]
[(' ', 0.18634420397310855), ('e', 0.10134871739747014), ('t', 0.07268976780147668), ('a', 0.0661844792596171), ('o', 0.06199839148549294), ('n', 0.05929195574383435), ('i', 0.05589620289558513), ('h', 0.05391469951731614), ('s', 0.0524923556707907), ('r', 0.047558671278950346), ('d', 0.03812516107809325), ('l', 0.031093414285732788), ('u', 0.021019153237556352), ('m', 0.019841907112820068), ('c', 0.019639223924127848), ('w', 0.019079740617322483), ('f', 0.017648007294004592), ('g', 0.016483064621617038), ('y', 0.014872283689118956), ('p', 0.0144776018887742), ('b', 0.01112782509930181), ('v', 0.00867328635

In [16]:
def prepare_text(text):
    text = ''.join(str(x).lower() for x in text)
    text = text.lower()
    text = ''.join(i for i in text if not i in bad_chars)
    text = text.replace("\n", " ")
    return text

In [38]:
original_sentence = "And if all others accepted the lie which the Party imposed, if all records told the same tale, then the lie passed into history and became truth. 'Who controls the past,' ran the Party slogan, 'controls the future: who controls the present controls the past.' And yet the past, though of its nature alterable, never had been altered. Whatever was true now was true from everlasting to everlasting. It was quite simple. All that was needed was an unending series of victories over your own memory. 'Reality control', they called it: in Newspeak, 'doublethink'."
original_sentence = prepare_text(original_sentence)
print(original_sentence)

and if all others accepted the lie which the party imposed if all records told the same tale then the lie passed into history and became truth who controls the past ran the party slogan controls the future who controls the present controls the past and yet the past though of its nature alterable never had been altered whatever was true now was true from everlasting to everlasting it was quite simple all that was needed was an unending series of victories over your own memory reality control they called it in newspeak doublethink


In [39]:
import random

def cipher_text(text):
    tokens_randomy_shuffled = tokens.copy()
    random.shuffle(tokens_randomy_shuffled)
    ciphered_text = ''.join(tokens_randomy_shuffled[tokens.index(x)] for x in text)
    mapping = {}
    for i in range(len(tokens_randomy_shuffled)):
        mapping[tokens[i]] = tokens_randomy_shuffled[i]
    print(mapping)
    return ciphered_text

ciphered_text = cipher_text(original_sentence)
print(ciphered_text)

{' ': 'x', 'a': 'b', 'b': 'a', 'c': 'w', 'd': 'v', 'e': 'y', 'f': 'j', 'g': 'p', 'h': ' ', 'i': 'c', 'j': 'k', 'k': 'l', 'l': 't', 'm': 'm', 'n': 'q', 'o': 'r', 'p': 'h', 'q': 'e', 'r': 'f', 's': 'g', 't': 'z', 'u': 'i', 'v': 'o', 'w': 'u', 'x': 'd', 'y': 's', 'z': 'n'}
bqvxcjxbttxrz yfgxbwwyhzyvxz yxtcyxu cw xz yxhbfzsxcmhrgyvxcjxbttxfywrfvgxzrtvxz yxgbmyxzbtyxz yqxz yxtcyxhbggyvxcqzrx cgzrfsxbqvxaywbmyxzfiz xu rxwrqzfrtgxz yxhbgzxfbqxz yxhbfzsxgtrpbqxwrqzfrtgxz yxjizifyxu rxwrqzfrtgxz yxhfygyqzxwrqzfrtgxz yxhbgzxbqvxsyzxz yxhbgzxz rip xrjxczgxqbzifyxbtzyfbatyxqyoyfx bvxayyqxbtzyfyvxu bzyoyfxubgxzfiyxqruxubgxzfiyxjfrmxyoyftbgzcqpxzrxyoyftbgzcqpxczxubgxeiczyxgcmhtyxbttxz bzxubgxqyyvyvxubgxbqxiqyqvcqpxgyfcygxrjxocwzrfcygxroyfxsrifxruqxmymrfsxfybtczsxwrqzfrtxz ysxwbttyvxczxcqxqyughyblxvriatyz cql


In [40]:
def decipher_text(text):
    frequencies_dict_cipher = {}
    for token in text:
        frequencies_dict_cipher[token] = frequencies_dict_cipher.get(token, 0) + 1
    frequencies_dict_counter_cipher = Counter(frequencies_dict_cipher)
    frequencies_dict_normalized_cipher = [(item[0], item[1]/len(text)) for item in frequencies_dict_counter_cipher.most_common()]
    mapping_freq_ciphered_original = {}
    for i in range(len(frequencies_dict_normalized_cipher)):
        mapping_freq_ciphered_original[frequencies_dict_normalized_cipher[i][0]] = frequencies_dict_normalized[i][0]
    print(mapping_freq_ciphered_original)
    deciphered_text = ''.join(mapping_freq_ciphered_original[x] for x in text)
    return deciphered_text

deciphered_text = decipher_text(ciphered_text)
print(deciphered_text)

{'x': ' ', 'y': 'e', 'z': 't', 'b': 'a', 'r': 'o', 'f': 'n', 'g': 'i', 'q': 'h', 't': 's', ' ': 'r', 'c': 'd', 'v': 'l', 'w': 'u', 'u': 'm', 'h': 'c', 'i': 'w', 's': 'f', 'm': 'g', 'j': 'y', 'o': 'p', 'p': 'b', 'a': 'v', 'l': 'k', 'e': 'x'}
ahl dy ass otreni auuectel tre sde mrdur tre cantf dgcoiel dy ass neuonli tosl tre iage tase treh tre sde caiiel dhto rditonf ahl veuage tnwtr mro uohtnosi tre cait nah tre cantf isobah uohtnosi tre ywtwne mro uohtnosi tre cneieht uohtnosi tre cait ahl fet tre cait trowbr oy dti hatwne astenavse hepen ral veeh astenel mratepen mai tnwe hom mai tnwe ynog epensaitdhb to epensaitdhb dt mai xwdte idgcse ass trat mai heelel mai ah whehldhb iendei oy pdutondei open fown omh gegonf neasdtf uohtnos tref uassel dt dh hemiceak lowvsetrdhk


In [41]:
def get_accuracy(original, deciphered):
    num_matches = 0
    for i in range(min(len(original), len(deciphered))):
        if original[i] == deciphered[i]:
            num_matches+=1
    return num_matches / len(original)

print(get_accuracy(original_sentence, deciphered_text))

0.5149812734082397


### Вывод ###
Какие-то буквы были дешифрованы верно, но текст все еще нечитабельный (тяжело предпложить, что это за слова).

### Задание 2 ###
Вряд ли в результате получилась такая уж хорошая расшифровка, разве что если вы брали в качестве тестовых данных целые рассказы. Но и Шерлок Холмс был не так уж прост: после буквы E, которая действительно выделяется частотой, дальше он анализировал уже конкретные слова и пытался угадать, какими они могли бы быть. Я не знаю, как запрограммировать такой интуитивный анализ, так что давайте просто сделаем следующий логический шаг:
- подсчитайте частоты биграмм (т.е. пар последовательных букв) по корпусам;
- проведите тестирование аналогично п.1, но при помощи биграмм.

In [42]:
def get_bigrams_freq(text, tokens):
    bigrams_frequencies_dict = {}
    for i in range(len(tokens)):
        for j in range(len(tokens)):
            bigram = str(tokens[i]) + str(tokens[j])
            bigrams_frequencies_dict[bigram] = 0
    
    for i in range(len(text) - 1):
        bigram = str(text[i:i+2])
        bigrams_frequencies_dict[bigram] += 1
    bigram_frequencies_dict_counter = Counter(bigrams_frequencies_dict)    
    bigram_frequencies_dict_normalized = [(item[0], item[1]/(len(text)-1)) for item in bigram_frequencies_dict_counter.most_common()]
    return bigram_frequencies_dict_normalized

bigram_frequencies_dict_normalized = get_bigrams_freq(text, tokens)
bigram_tokens = [item[0] for item in bigram_frequencies_dict_normalized]
print(bigram_frequencies_dict_normalized)
print(bigram_tokens)

assert len(bigram_frequencies_dict_normalized) == len(tokens)*len(tokens)

[('e ', 0.0358027341517722), (' t', 0.02821571067192089), ('d ', 0.024486468302573075), ('he', 0.02429349830325624), ('th', 0.0237715727346342), (' a', 0.022398118477751628), ('s ', 0.02023400526393666), ('t ', 0.018756295403396336), (' h', 0.016112023617455756), ('in', 0.015601753954832718), ('an', 0.014742778437739492), ('er', 0.014111093087626835), (' s', 0.013168907352707392), (' w', 0.013088611111380924), ('n ', 0.01305461471888383), ('re', 0.011485276486184675), ('nd', 0.01114887408804677), (' o', 0.010947162159230684), ('y ', 0.010099518772969825), ('r ', 0.010013718353810494), (' i', 0.009886474713321373), ('ed', 0.009249608960542492), ('at', 0.009065057115558271), ('on', 0.00889993178057239), ('ha', 0.008850394180076625), ('o ', 0.0086732891639251), ('en', 0.008540865120769755), ('hi', 0.008390957218615906), (' b', 0.008119309853805799), ('ng', 0.008062972974810616), ('to', 0.007983000508650787), ('ou', 0.007574720023519028), ('g ', 0.00712499631705748), ('is', 0.0070728685152

In [43]:
def cipher_text_bigram(text, tokens):
    tokens_randomly_shuffled = tokens.copy()
    random.shuffle(tokens_randomly_shuffled)
    ciphered_text = "" #''.join(tokens_randomy_shuffled[tokens.index(x)] for x in text)
    
    for i in range(0,len(text) - 1,2):
        bigram = str(text[i:i+2])
        ciphered_text += tokens_randomly_shuffled[tokens.index(bigram)]
        
    return ciphered_text

bigram_ciphered_text = cipher_text_bigram(original_sentence, bigram_tokens)
print(bigram_ciphered_text)

ikzrybz mupfayqhhfetletidewyfpqruivcnjrnwyfpnipdwnoiwxlhdeoiudhltpkelwhthfy  rwyfpijfbf ebfawyfprmayf ckf xmcbdeoicyjrnjsnmazpikzrvvifwiwyqgayvcajfgfndohmhfayf xmsnkvikwyfpnipdwnijubwarmlwcywjeewyfplcwpndf ysjrlwcywjeewyfpnikegdcyfgfndohmhfayf xmsnz mdxempwyfpni bimayxpw pfudxthfzduhkez tmqhkdfakjvdqhzchnhwhurmhl ekezryskovdqhvc bwyqgf oyiztzhfdoaklcwjcjvdqhgvsnzfsfy ytanwl begsloiimtzhfevxtf fswxfaz muwyisimtzhfugdedevc bz rmchumzjslijqhuihfporv ay ep fpfankgkpndpfsrzwqsmazpkehlxtzplwcywjtpayfwfghlfazrxtoirmugvfjeceqjxpgrmpnjhc


In [44]:
def bigram_decipher_text(text, bigram_frequencies_dict_normalized):    
    bigram_frequencies_dict_normalized_cipher = get_bigrams_freq(text, tokens)
    bigram_tokens_cipher = [item[0] for item in bigram_frequencies_dict_normalized_cipher]
        
    mapping_freq_ciphered_original = {}
    for i in range(len(bigram_frequencies_dict_normalized_cipher)):
        mapping_freq_ciphered_original[bigram_frequencies_dict_normalized_cipher[i][0]] = bigram_frequencies_dict_normalized[i][0]
    
    deciphered_text = ""
    for i in range(0,len(text) - 1,2):
        bigram = str(text[i:i+2])
        deciphered_text += mapping_freq_ciphered_original[bigram]
    
    return deciphered_text

bigram_deciphered_text = bigram_decipher_text(bigram_ciphered_text, bigram_frequencies_dict_normalized)
print(bigram_deciphered_text)

r ngyarestonth hhem odvaer td scdienselo td  widlan iotlern ety fo s iboheofne td  mpa ados  td inth atr aleryern antesehaed rr ngysu rc tiethen yasadislyheth alehatyr  td  widla msmnkin ianhil  td ma qat aacte ianhil  td  w sutanasadislyheth alehareupoanc td  wndf thoovoonet bhergcu srept hugs ba d hddplctoriny to sngacp  d hennd tie aauivo heisvemahifr d hshhahrknofuaalocndcourn f o hewa b a cios rest tdrf o heweererenndreinriafrlur m hdihedsraovofpo nonalewexatonbrguwned r sy  b r ianhifothchasy s ng bn inwels pkeoioottncseig


In [45]:
print(get_accuracy(original_sentence, bigram_deciphered_text))

0.11610486891385768


### Вывод ###
Метрика качества значительно ухудшилась, текст все так же невозможно прочесть. Возможно несправедливо их сравнивать по одинаковой метрике, так как одна ошибочная биграмма дает двойное ухудшение, потому что последующий символ тоже будет декодирован неверно, но с другой стороны справедливо и обратное. Но так как текст для шифрования довольно мал, то вероятность того, что там встретятся биграммы в нужном объеме очень мала, в то время как там скорее всего встретятся все буквы (1-граммы). Получается, что частота встречаемости большинства биграмм равна нулю, соответственно в хвосте (который занимает почти весь массив) отсортированного массива частот почти все будет случайным. Но все-таки даже при таком раскладе accuracy ненулевая.

### Задание 3 ###
Но и это ещё не всё: биграммы скорее всего тоже далеко не всегда работают. Основная часть задания — в том, как можно их улучшить:
- предложите метод обучения перестановки символов в этом задании, основанный на MCMC-сэмплировании, но по-прежнему работающий на основе статистики биграмм;
- реализуйте и протестируйте его, убедитесь, что результаты улучшились.

### Решение ###

Заимствовано из https://towardsdatascience.com/applications-of-mcmc-for-cryptography-and-optimization-1f99222b7132

#### Общие концепции ####
Markov Chain Monte Carlo (MCMC) - класс алгоритмов для сэмплирования из произвольного распределения путем построения марковской цепи так, что ее стационарные вероятности и будут искомым распределением (то есть стационарная вероятность оказаться в каком-то состоянии C марковской цепи - это вероятность встретить C при заданном распределении - заданной функции плотности/вероятности).

#### Что такое состояния в нашем случае? ####
Множество состояний - это множество всевозможных перестановок символов в алфавите A. Тогда всего состояний будет |A|!, где |A| - количество символов в алфавите А. Слишком много (для обычного языкового алфавита), но для MCMC-алгоритма не обязательно строить матрицу переходов явным образом. 

#### Схема работы алгоритма MCMC (Metropolis-Hastings Algorithm) ####
1. Начать в случайном состоянии $C_{i}$
2. Случайным образом выбрать новое состояние $C_{j}$.
3. Посчитаем вероятность перехода в новое состояние: $aij=min(sj*pji/si*pij,1)$
4. Перейдем в состояние $j$ из состояния $i$ с вероятностью $a_{ij}$ (подбросим монетку).
5. Повторим шаги 2-5, пока алгоритм не сойдется (долго).

В конечном итоге цепь Маркова сойдется и мы получим стационарные распределения S. Теперь можем использовать состояния цепи для сэмплинга.

#### Применим алгоритм Метрополиса-Гастингса к нашей задаче ####
1. Начнем в случайном состоянии
2. Выберем новое состояние. Правильнее всего это сделать перестановкой каких-то случайных букв, так как если наш алгоритм будет уже близо к какой-то "хорошей" перестановке, то выгодно сильно от нее не отклоняться.
3. Посчитаем вероятность перехода в новое состояние. Необходима такая функция оценки состояния, чтобы она стремила наш алгоритм к тому, чтобы вероятность сочетания букв AB в исходном тексте была такой же, как и вероятность сочетания A'B' в зашифрованном тексте. А чтобы понять, какое состояние лучше, нужно просто сравнить правдоподобия обоих шифров (то есть функций оценки состояния). 

Введем следующую функцию: 

$\prod_{(A,B) \in T} F(A,B)^{f(A,B)}$

здесь F(A,B) - частота сочетания букв AB в исходном тесте, а f(A,B) - в зашифрованном (имеются в виду буквы A и B, которые стоят в перестановке на месте исходных букв A,B).

Тогда получится, что если частоты сочетаний букв близки к друг другу, то значение функции будет выше. На практике, как и для оценок максимального правдоподобия будем использовать логарифмическую шкалу (то есть произведение заменим на сумму и будем брать экспоненту от соотношения двух функций).
4. Перейдем в состояние $j$ из состояния $i$ с вероятностью $aij=min(sj/si,1)$ (подбросим монетку). Здесь sj и si как раз и есть значения функции оценки состояний.
5. Повторим шаги 2-5, пока алгоритм не сойдется.

In [65]:
import string
import math
import random


def create_cipher_dict(cipher):
    cipher_dict = {}    
    for i in range(len(cipher)):
        cipher_dict[tokens[i]] = cipher[i]
    return cipher_dict


def apply_cipher_on_text(text,cipher):
    cipher_dict = create_cipher_dict(cipher) 
    text = list(text)
    newtext = "".join(cipher_dict[letter] for letter in text)    
    return newtext


def score_params_on_cipher(text):
    bigrams_frequencies_dict = {}
    for i in range(len(tokens)):
        for j in range(len(tokens)):
            bigram = str(tokens[i]) + str(tokens[j])
            bigrams_frequencies_dict[bigram] = 1    
    for i in range(len(text) - 1):
        bigram = str(text[i:i+2])
        bigrams_frequencies_dict[bigram] += 1
    return bigrams_frequencies_dict


def get_cipher_score(text,cipher,scoring_params):
    cipher_dict = create_cipher_dict(cipher)
    decrypted_text = apply_cipher_on_text(text,cipher)
    scored_f = score_params_on_cipher(decrypted_text)
    cipher_score = 0
    for k,v in scored_f.items():
        if k in scoring_params:
            cipher_score += v*math.log(scoring_params[k])
    return cipher_score


def generate_cipher(cipher):    
    pos1 = random.randint(0, len(list(cipher))-1)
    pos2 = random.randint(0, len(list(cipher))-1)
    if pos1 == pos2:
        return generate_cipher(cipher)
    else:        
        cipher = list(cipher)
        pos1_alpha = cipher[pos1]
        pos2_alpha = cipher[pos2]
        cipher[pos1] = pos2_alpha
        cipher[pos2] = pos1_alpha
        return "".join(cipher)


def random_coin(p):
    unif = random.uniform(0,1)
    if unif>=p:
        return False
    else:
        return True


def MCMC_decrypt(n_iter,cipher_text,scoring_params):
    current_cipher = "".join(x for x in tokens) 
    state_keeper = set()
    best_state = ''
    score = 0
    for i in range(n_iter):
        state_keeper.add(current_cipher)
        proposed_cipher = generate_cipher(current_cipher)
        score_current_cipher = get_cipher_score(cipher_text,current_cipher,scoring_params)
        score_proposed_cipher = get_cipher_score(cipher_text,proposed_cipher,scoring_params)
        acceptance_probability = min(1,math.exp(score_proposed_cipher-score_current_cipher))
        if score_current_cipher>score:
            best_state = current_cipher
        if random_coin(acceptance_probability):
            current_cipher = proposed_cipher
        if i%500==0:
            print("iter",i,":",apply_cipher_on_text(cipher_text,current_cipher)[0:99])
    return state_keeper,best_state



scoring_params = score_params_on_cipher(text)
print(original_sentence)

encryption_key = "xebprohyauftidsjlkzmwvngqc"
cipher_text = apply_cipher_on_text(original_sentence,encryption_key)
decryption_key = "icznbkxgmprqtwfdyeoljvuahs"

print("Text To Decode:", cipher_text)
print("\n")
states,best_state = MCMC_decrypt(10000,cipher_text,scoring_params)
print("\n")
print("Decoded Text:",apply_cipher_on_text(cipher_text,best_state))
print("\n")
print("MCMC KEY FOUND:",best_state)
print("ACTUAL DECRYPTION KEY:",decryption_key)

and if all others accepted the lie which the party imposed if all records told the same tale then the lie passed into history and became truth who controls the past ran the party slogan controls the future who controls the present controls the past and yet the past though of its nature alterable never had been altered whatever was true now was true from everlasting to everlasting it was quite simple all that was needed was an unending series of victories over your own memory reality control they called it in newspeak doublethink
Text To Decode: esrxuhxeiixjwaozmxeppolworxwaoxiuoxgaupaxwaoxlezwcxudljmorxuhxeiixzopjzrmxwjirxwaoxmedoxweioxwaosxwaoxiuoxlemmorxuswjxaumwjzcxesrxbopedoxwzvwaxgajxpjswzjimxwaoxlemwxzesxwaoxlezwcxmijyesxpjswzjimxwaoxhvwvzoxgajxpjswzjimxwaoxlzomoswxpjswzjimxwaoxlemwxesrxcowxwaoxlemwxwajvyaxjhxuwmxsewvzoxeiwozebioxsonozxaerxboosxeiwozorxgaewonozxgemxwzvoxsjgxgemxwzvoxhzjdxonoziemwusyxwjxonoziemwusyxuwxgemxkvuwoxmudlioxeiixwaewxgemxsoororxgemxesxvsosrusyxmozuomxjhx

### Задание 4 ###
Расшифруйте сообщение:
←⇠⇒↟↹↷⇊↹↷↟↤↟↨←↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↟⇒↟↹⇷⇛⇞↨↟↹↝⇛⇯↳⇴⇒⇈↝⇊↾↹↨←⇌⇠↨↹⇙↹⇸↨⇛↙⇛↹⇠⇛⇛↲⇆←↝↟↞↹⇌⇛↨⇛⇯⇊↾↹⇒←↙⇌⇛↹⇷⇯⇛⇞↟↨⇴↨⇈↹⇠⇌⇛⇯←←↹↷⇠←↙⇛↹↷⇊↹↷⇠←↹⇠↤←⇒⇴⇒↟↹⇷⇯⇴↷↟⇒⇈↝⇛↹↟↹⇷⇛⇒⇙⇞↟↨←↹↳⇴⇌⇠↟↳⇴⇒⇈↝⇊↾↹↲⇴⇒⇒↹⇰⇴↹⇷⇛⇠⇒←↤↝←←↹⇞←↨↷←⇯↨⇛←↹⇰⇴↤⇴↝↟←↹⇌⇙⇯⇠⇴↹↘⇛↨↞↹⇌⇛↝←⇞↝⇛↹↞↹↝↟⇞←↙⇛↹↝←↹⇛↲←⇆⇴⇏
Или это (они одинаковые, второй вариант просто на случай проблем с юникодом):
დჳჵჂႨშႼႨშჂხჂჲდႨსႹႭჾႣჵისႼჰႨჂჵჂႨႲႹႧჲჂႨსႹႭჾႣჵისႼჰႨჲდႩჳჲႨჇႨႠჲႹქႹႨჳႹႹჱჶდსჂႽႨႩႹჲႹႭႼჰႨჵდქႩႹႨႲႭႹႧჂჲႣჲიႨჳႩႹႭდდႨშჳდქႹႨშႼႨშჳდႨჳხდჵႣჵჂႨႲႭႣშჂჵისႹႨჂႨႲႹჵჇႧჂჲდႨჾႣႩჳჂჾႣჵისႼჰႨჱႣჵჵႨეႣႨႲႹჳჵდხსდდႨႧდჲშდႭჲႹდႨეႣხႣსჂდႨႩჇႭჳႣႨႾႹჲႽႨႩႹსდႧსႹႨႽႨსჂႧდქႹႨსდႨႹჱდჶႣნ

### Решение 4 ###
Кажется, текс на русском. Поэтому взяла случайный текст из https://cryptii.com/pipes/caesar-cipher

In [56]:
cipher_text = "pm ol ohk hufaopun jvumpkluaphs av zhf ol dyval pa pu jpwoly aoha pz if zv johunpun aol vykly vm aol slaalyz vm aol hswohila aoha uva h dvyk jvbsk il thkl vba"

states,best_state = MCMC_decrypt(20000,cipher_text,scoring_params)
print("\n")
print("Decoded Text:",apply_cipher_on_text(cipher_text,best_state))
print("\n")

iter 0 : pm ol ohk hufaopun jvumpkluaphs av zhf ol dyval pa pu jpwoly aoha pz if zv johunpun aol vykly vm ao
iter 500 : sl th tod oikatsin prilsdhiasof ar mok th berah sa si psuthe atoa sm ck mr ptoinsin ath redhe rl at
iter 1000 : ad th ton oilstaim pridanhisaow sr fol th bersh as ai pauthe stos af cl fr ptoimaim sth renhe rd st
iter 1500 : od th tan airstoig plidonhisoaw sl far th belsh os oi pouthe stas of cr fl ptaigoig sth lenhe ld st
iter 2000 : od th taw airstoin plidowhisoac sl far th belsh os oi pouthe stas of gr fl ptainoin sth lewhe ld st
iter 2500 : od th taw ailstoin pridowhisoak sr fal th bersh os oi pouthe stas of gl fr ptainoin sth rewhe rd st
iter 3000 : od th taw ailstoin pridowhisoac sr fal th bersh os oi pouthe stas of gl fr ptainoin sth rewhe rd st
iter 3500 : od th taw ailstoin pridowhisoac sr fal th bersh os oi pouthe stas of gl fr ptainoin sth rewhe rd st
iter 4000 : od th taw ailstoin pridowhisoac sr fal th bersh os oi pouthe stas of gl fr ptainoin sth rewhe rd

### Бонус 1 ###
Бонус: а что если от биграмм перейти к триграммам (тройкам букв) или даже больше? Улучшатся ли результаты? Когда улучшатся, а когда нет? Чтобы ответить на этот вопрос эмпирически, уже может понадобиться погенерировать много тестовых перестановок и последить за метриками, глазами может быть и не видно.

Посмотрим на 3-граммы.
Кажется, переход к большиму количеству букв дает большую скорость сходимости. 

Но из-за того, что текст слишком мал, функция (правдоподобия) может легко уйти в локальный максимум, откуда не выберется.

Думаю, что переход к 4-кам букв не имеет большого смысла: это будет фактически попытка запомнить слова.

In [78]:
import string
import math
import random


def score_params_on_cipher(text):
    trigrams_frequencies_dict = {}
    for i in range(len(tokens)):
        for j in range(len(tokens)):
            for k in range(len(tokens)):
                trigram = str(tokens[i]) + str(tokens[j]) + str(tokens[k])
                trigrams_frequencies_dict[trigram] = 1    
    for i in range(len(text) - 2):
        trigram = str(text[i:i+3])
        trigrams_frequencies_dict[trigram] += 1
    return trigrams_frequencies_dict


scoring_params = score_params_on_cipher(text)
print(original_sentence)

encryption_key = "xebprohyauftidsjlkzmwvngqc"
cipher_text = apply_cipher_on_text(original_sentence,encryption_key)
decryption_key = "icznbkxgmprqtwfdyeoljvuahs"

print("Text To Decode:", cipher_text)
print("\n")
states,best_state = MCMC_decrypt(3000,cipher_text,scoring_params)
print("\n")
print("Decoded Text:",apply_cipher_on_text(cipher_text,best_state))
print("\n")
print("MCMC KEY FOUND:",best_state)
print("ACTUAL DECRYPTION KEY:",decryption_key)

and if all others accepted the lie which the party imposed if all records told the same tale then the lie passed into history and became truth who controls the past ran the party slogan controls the future who controls the present controls the past and yet the past though of its nature alterable never had been altered whatever was true now was true from everlasting to everlasting it was quite simple all that was needed was an unending series of victories over your own memory reality control they called it in newspeak doublethink
Text To Decode: esrxuhxeiixjwaozmxeppolworxwaoxiuoxgaupaxwaoxlezwcxudljmorxuhxeiixzopjzrmxwjirxwaoxmedoxweioxwaosxwaoxiuoxlemmorxuswjxaumwjzcxesrxbopedoxwzvwaxgajxpjswzjimxwaoxlemwxzesxwaoxlezwcxmijyesxpjswzjimxwaoxhvwvzoxgajxpjswzjimxwaoxlzomoswxpjswzjimxwaoxlemwxesrxcowxwaoxlemwxwajvyaxjhxuwmxsewvzoxeiwozebioxsonozxaerxboosxeiwozorxgaewonozxgemxwzvoxsjgxgemxwzvoxhzjdxonoziemwusyxwjxonoziemwusyxuwxgemxkvuwoxmudlioxeiixwaewxgemxsoororxgemxesxvsosrusyxmozuomxjhx

### Бонус 2 ###
Бонус: какие вы можете придумать применения для этой модели? Пляшущие человечки ведь не так часто встречаются в жизни (хотя встречаются! и это самое потрясающее во всей этой истории, но об этом я расскажу потом).

### Решение ###

Когда-то я занималась теорией кодирования, а конкретно низкоплотностными кодами с проверкой на четность (low-density parity-check codes). Суть: кодируем сообщение (с добавлением избыточности) и декодируем с помощью проверки на четность (то есть если какие-то биты были искажены, то уравнения проверки на четность не будут выполнены). 

Для декодирования этих кодов используется алгоритм распространения доверия (message passing algorithm, belief propagation), на практике часто используется его модификация min-sum. Возникла идея применения MCMC-метода для декодирования кодовых слов, где функцию, которую надо максимизировать можно сделать такой, чтобы она возрастала, если почти все уравнения на четность выполняются. Правда кажется, что на практике это будет долго выполняться (современная скорость передачи информации требует большой скорости декодирования, а LDPC как раз выигрывают - достигают пределов пропускной способности - тогда, когда кодовые слова достаточно большие). 